In [5]:
import pandas as pd
import numpy as np
import joblib
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score

In [6]:
label_to_coordinates = {
    "1-1": (0, 0), "1-2": (0.6, 0), "1-3": (1.2, 0), "1-4": (1.8, 0), "1-5": (2.4, 0), "1-6": (3.0, 0),"1-7": (3.6, 0), "1-8": (4.2, 0), "1-9": (4.8, 0), "1-10": (5.4, 0), "1-11": (6.0, 0),
    "2-1": (0, 0.6), "2-11": (6.0, 0.6),
    "3-1": (0, 1.2), "3-11": (6.0, 1.2),
    "4-1": (0, 1.8), "4-11": (6.0, 1.8),
    "5-1": (0, 2.4), "5-11": (6.0, 2.4),
    "6-1": (0, 3.0), "6-2": (0.6, 3.0), "6-3": (1.2, 3.0), "6-4": (1.8, 3.0), "6-5": (2.4, 3.0),"6-6": (3.0, 3.0), "6-7": (3.6, 3.0), "6-8": (4.2, 3.0), "6-9": (4.8, 3.0), "6-10": (5.4, 3.0), "6-11": (6.0, 3.0),
    "7-1": (0, 3.6), "7-11": (6.0, 3.6),
    "8-1": (0, 4.2), "8-11": (6.0, 4.2),
    "9-1": (0, 4.8), "9-11": (6.0, 4.8),
    "10-1": (0, 5.4), "10-11": (6.0, 5.4),
    "11-1": (0, 6.0), "11-2": (0.6, 6.0), "11-3": (1.2, 6.0), "11-4": (1.8, 6.0), "11-5": (2.4, 6.0),"11-6": (3.0, 6.0), "11-7": (3.6, 6.0), "11-8": (4.2, 6.0), "11-9": (4.8, 6.0), "11-10": (5.4, 6.0), "11-11": (6.0, 6.0)
}
label_mapping = {
    '11': '1-1','10': '1-2','9': '1-3','8': '1-4','7': '1-5','6': '1-6','5': '1-7','4': '1-8','3': '1-9','2': '1-10','1': '1-11',
    '12': '2-1','30': '2-11',
    '13': '3-1','29': '3-11',
    '14': '4-1','28': '4-11',
    '15': '5-1','27': '5-11',
    '16': '6-1','17': '6-2','18': '6-3','19': '6-4','20': '6-5','21': '6-6','22': '6-7','23': '6-8','24': '6-9','25': '6-10','26': '6-11',
    '49': '7-1','31': '7-11',
    '48': '8-1','32': '8-11',
    '47': '9-1','33': '9-11',
    '46': '10-1','34': '10-11',
    '45': '11-1','44': '11-2','43': '11-3','42': '11-4','41': '11-5','40': '11-6','39': '11-7','38': '11-8','37': '11-9','36': '11-10','35': '11-11'
}

In [7]:
reverse_label_mapping = {v:int(k)-1 for k,v in label_mapping.items()}
all_date = ["2024_12_21","2024_12_27","2025_01_03","2025_01_10","2025_02_28"]

In [ ]:
date_mde = []
date_accuracy = []

for date in all_date:
    # --- 3. 讀取「別周」測試資料，只取你做回歸與 DNN 時用到的欄位 ---
    file_test = f"timestamp_allignment_Balanced_{date}_rtt_logs.csv"

    cols = ['Label',
            'AP1_Distance (mm)',
            'AP1_StdDev (mm)',
            'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi'
            ]

    df = pd.read_csv(file_test, usecols=cols)

    # --- 4. 填補缺失值（group by Label 填平均） ---
    df = df.groupby('Label').apply(lambda g: g.fillna(g.mean())).reset_index()

    
    allmde = []
    allaccuracy = []

    for loop in range(10):
        # --- 2. 載入已訓練好的模型和標準化器 ---
        regressor = joblib.load(f'timedegrade\\regressor_model_AP1&AP4_{loop}.pkl')        # 回歸器
        reg_scaler = joblib.load(f'timedegrade\\scaler_regressor_input_{loop}.pkl')        # 回歸 sclaer
        scaler_dnn = joblib.load(f'timedegrade\\scaler_regressor_dnn_AP1&AP4_{loop}.pkl')  # DNN 特徵標準化
        dnn = load_model(f'timedegrade\\regressor_dnn_AP1&AP4_{loop}.h5')                  # 分類 DNN

        # mask_ap1 = df['AP1_Rssi'].notna()
        # X_ap1 = df.loc[mask_ap1, ['AP1_Rssi']].rename(columns={'AP1_Rssi':'Rssi'})
        # df.loc[mask_ap1, 'AP1_Distance_predicted'] = regressor.predict(X_ap1)

        mask_ap2 = df['AP2_Rssi'].notna()
        X_ap2 = reg_scaler.transform(df.loc[mask_ap2, ['AP2_Rssi']].rename(columns={'AP2_Rssi': 'Rssi'}))
        df.loc[mask_ap2, 'AP2_Distance_predicted'] = regressor.predict(X_ap2)

        mask_ap3 = df['AP3_Rssi'].notna()
        X_ap3 = reg_scaler.transform(df.loc[mask_ap3, ['AP3_Rssi']].rename(columns={'AP3_Rssi': 'Rssi'}))
        df.loc[mask_ap3, 'AP3_Distance_predicted'] = regressor.predict(X_ap3)

        mask_ap4 = df['AP4_Rssi'].notna()
        X_ap4 = df.loc[mask_ap4, ['AP4_Rssi']].rename(columns={'AP4_Rssi':'Rssi'})
        df.loc[mask_ap4, 'AP4_Distance_predicted'] = regressor.predict(X_ap4)


        # --- 6. 準備 DNN 輸入特徵，順序必須跟訓練時一致 ---
        X_test = df[[   'AP1_Distance (mm)',
                        'AP1_StdDev (mm)',
                        'AP1_Rssi', 'AP2_Rssi', 'AP3_Rssi', 'AP4_Rssi',
                        'AP2_Distance_predicted', 'AP3_Distance_predicted', 'AP4_Distance_predicted']].values

        # --- 7. 標準化 + 預測 + 評估 Accuracy ---
        X_test_scaled = scaler_dnn.transform(X_test)
        y_true = df['Label'].map(reverse_label_mapping).values

        y_prob = dnn.predict(X_test_scaled,verbose=0)
        y_pred = np.argmax(y_prob, axis=1)
        acc = accuracy_score(y_true, y_pred)
        print(f"Test Accuracy: {acc:.4f}")

        # --- 8. 計算 MDE ---
        # 先將 numeric label 轉回字串，再對應座標
        y_pred_labels = [ label_mapping[str(i+1)] for i in y_pred ]
        y_true_labels = [ label_mapping[str(i+1)] for i in y_true ]

        coords_pred = np.array([ label_to_coordinates[l] for l in y_pred_labels ])
        coords_true = np.array([ label_to_coordinates[l] for l in y_true_labels ])

        distances = np.linalg.norm(coords_pred - coords_true, axis=1)
        mde = distances.mean()
        print(f"Test MDE: {mde:.4f} m")

        allmde.append(mde)
        allaccuracy.append(acc)

    # print([round(float(mde), 4) for mde in allmde])
    mmmm = round(sum(map(float, allmde)) / len(allmde), 4)
    print("平均 MDE:", mmmm)
    date_mde.append(mmmm)

    # print([round(float(acc), 4) for acc in allaccuracy])
    aaaa = round(sum(map(float, allaccuracy)) / len(allaccuracy), 4)
    print("平均 Accuracy:", aaaa)
    date_accuracy.append(aaaa)


print(all_date)
print("date MDE:" , date_mde)
print("date Accuracy:" , date_accuracy)




c:\Users\吳定洋\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SGDRegressor was fitted without feature names
  warnings.warn(
c:\Users\吳定洋\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Test Accuracy: 0.2921
Test MDE: 1.2488 m


c:\Users\吳定洋\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SGDRegressor was fitted without feature names
  warnings.warn(
c:\Users\吳定洋\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Test Accuracy: 0.2976
Test MDE: 1.1867 m


c:\Users\吳定洋\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SGDRegressor was fitted without feature names
  warnings.warn(
c:\Users\吳定洋\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


...

Test Accuracy: 0.1561
Test MDE: 1.5576 m


c:\Users\吳定洋\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SGDRegressor was fitted without feature names
  warnings.warn(
c:\Users\吳定洋\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Test Accuracy: 0.1291
Test MDE: 1.6039 m


c:\Users\吳定洋\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SGDRegressor was fitted without feature names
  warnings.warn(
c:\Users\吳定洋\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Test Accuracy: 0.1120
Test MDE: 1.5298 m
[1.5924, 1.6474, 1.6205, 1.6979, 1.583, 1.6632, 1.5869, 1.5576, 1.6039, 1.5298]
平均 MDE: 1.6083
[0.155, 0.1119, 0.116, 0.1363, 0.1534, 0.1282, 0.1555, 0.1561, 0.1291, 0.112]
平均 Accuracy: 0.1354
